In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import scipy.stats as st
import scipy.optimize as sco
import MCF_PETNICA as mp

In [ ]:
filenames = ['AUDUSD_D1.xlsx', 'BTCUSD_D1.xlsx', 'ETHUSD_D1.xlsx', 'EURUSD_D1.xlsx', 'GBPUSD_D1.xlsx', 'NZDUSD_D1.xlsx', 'USDCAD_D1.xlsx', 'USDCHF_D1.xlsx', 'USDJPY_D1.xlsx', 'XAGUSD_D1.xlsx', 'XAUUSD_D1.xlsx', ]

header = []
data = []
for name in filenames:
  data.append(pd.read_excel(name))



In [ ]:
import pandas as pd

filenames = ['AUDUSD_D1.xlsx', 'BTCUSD_D1.xlsx', 'ETHUSD_D1.xlsx', 'EURUSD_D1.xlsx', 'GBPUSD_D1.xlsx', 'NZDUSD_D1.xlsx', 'USDCAD_D1.xlsx', 'USDCHF_D1.xlsx', 'USDJPY_D1.xlsx', 'XAGUSD_D1.xlsx', 'XAUUSD_D1.xlsx', ]


all_data = []

for name in filenames:
    df = pd.read_excel(name)
    if 'USD_' in name:
      df['Close']  = 1/df['Close']
      df['Open'] = 1/df['Open']
    df['Price'] = (df['Close'] - df['Open']) / df['Open']
    df = df[['Time', 'Price']]
    df.rename(columns={'Price': name.split('.')[0]}, inplace=True)
    all_data.append(df)

# Merging all dataframes on 'Date'
merged_data = all_data[0]
for df in all_data[1:]:
    merged_data = pd.merge(merged_data, df, on='Time', how='outer')

# Display the combined data
merged_data = merged_data.dropna()
# print(merged_data)


In [ ]:
retM = merged_data.drop("Time", axis = 'columns')

In [ ]:
criterium = lambda w: mp.pV(w,retM)
start_portfolio = [ 1/11 ] * 11
granice = [ (0,1) ] * 11

result=sco.minimize(criterium, start_portfolio,constraints=[dict(type='eq',fun=lambda w:sum(w)-1)], bounds = granice)
result.x

array([1.99965326e-17, 0.00000000e+00, 0.00000000e+00, 1.69516038e-01,
       2.08166817e-17, 2.07475249e-17, 3.79551235e-01, 1.96788409e-01,
       2.54144318e-01, 0.00000000e+00, 2.15389291e-17])

In [ ]:
print(f"Risk {mp.pV(result.x, retM)}, Expected Profit: {mp.per(result.x, retM)}")

Risk 0.0025080503887368708, Expected Profit: 0.0200915025620585


In [ ]:
criterium = lambda w: - mp.per(w, retM) / np.sqrt(mp.pV(w,retM))
start_portfolio = [ 1/11 ] * 11
granice = [ (0,1) ] * 11

result_sharpe=sco.minimize(criterium, start_portfolio,constraints=[dict(type='eq',fun=lambda w:sum(w)-1)], bounds = granice)
result_sharpe.x

array([0.00000000e+00, 0.00000000e+00, 4.06003156e-02, 0.00000000e+00,
       2.23754664e-16, 1.97191578e-16, 3.58061942e-02, 0.00000000e+00,
       9.23593490e-01, 0.00000000e+00, 3.32165504e-16])

In [ ]:
print(f"Risk {mp.pV(result_sharpe.x, retM)}, Expected Profit: {mp.per(result_sharpe.x, retM)}")

Risk 0.006130792990536765, Expected Profit: 0.0581718571658748


In [ ]:
criterium = lambda w: - mp.per(w, retM)
start_portfolio = [ 1/11 ] * 11
granice = [ (0,1) ] * 11

result_profit_max=sco.minimize(criterium, start_portfolio,constraints=[dict(type='eq',fun=lambda w:sum(w)-1)], bounds = granice)
result_profit_max.x

array([0.00000000e+00, 9.71445147e-17, 1.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

In [ ]:
print(f"Risk {mp.pV(result_profit_max.x, retM)}, Expected Profit: {mp.per(result_profit_max.x, retM)}")

Risk 0.6370975461864719, Expected Profit: 0.2608808502783416


In [ ]:
criterium = lambda w: - mp.per(w, retM) + mp.pV(w, retM)
start_portfolio = [ 1/11 ] * 11
granice = [ (0,1) ] * 11

result_maxp_minv=sco.minimize(criterium, start_portfolio,constraints=[dict(type='eq',fun=lambda w:sum(w)-1)], bounds = granice)
result_maxp_minv.x

array([3.20126960e-17, 2.97755374e-16, 1.70743220e-01, 1.65128697e-16,
       0.00000000e+00, 2.07907530e-16, 3.18771898e-16, 0.00000000e+00,
       8.29256780e-01, 2.07523958e-16, 1.98034086e-16])

In [ ]:
print(f"Risk {mp.pV(result_maxp_minv.x, retM)}, Expected Profit: {mp.per(result_maxp_minv.x, retM)}")

Risk 0.0229000260432247, Expected Profit: 0.08685768912848649


In [ ]:
def get_good_dict(portfolio: list, currencies: list):
  df_portfolio = dict()
  for weight, currency in zip(portfolio, currencies):
    if weight < 1e-10:
      weight = 0
    elif weight > 1 - 1e-10:
      weight = 1
    df_portfolio[currency] = weight

  return df_portfolio

In [ ]:
currencies = merged_data.columns[1:]

In [ ]:
portfolio = get_good_dict(result_maxp_minv.x, currencies)
for x in portfolio:
  print(x, portfolio[x])

AUDUSD_D1 0
BTCUSD_D1 0
ETHUSD_D1 0.1707432199013493
EURUSD_D1 0
GBPUSD_D1 0
NZDUSD_D1 0
USDCAD_D1 0
USDCHF_D1 0
USDJPY_D1 0.8292567800986493
XAGUSD_D1 0
XAUUSD_D1 0


In [ ]:
def exchange_from_usd(portfolio: dict, rates, budget):
  portfolio = np.array(list(portfolio.values())) * budget
  return portfolio * rates

In [ ]:
def exchange_to_usd(money, rates):
  money = np.array(money)
  return np.sum(money / rates)

In [ ]:
def get_rates(df, day):
  rates = []
  for valuta in df:
    rates.append(valuta.iloc[-1]['Closed'])
  return rates


In [ ]:
exchange_rate_30724 = {
    "AUD": 1/0.65456,
    "BTC": 1/66568.2,
    "ETH": 1/3333.5,
    "EUR": 1/1.08317,
    "GBP": 1/1.28618,
    "NZD": 1/0.5895,
    "CAD": 1/1.38424,
    "CHF":  0.88581,
    "JPY":  154.809,
    "XAG":  27.806,
    "XAU": 1/2389.025,
}

In [ ]:
exchange_rate_7824 = {
    "AUD": 1/0.65172,
    "BTC": 1/54770.6,
    "ETH": 1/2365.6,
    "EUR": 1/1.09221,
    "GBP": 1/1.26838,
    "NZD": 1/0.59953,
    "CAD": 1/1.37458,
    "CHF":  0.86174,
    "JPY":  146.841,
    "XAG":  26.61,
    "XAU": 1/2386.125,
}

In [ ]:
def invest_portfolio(portfolio, budget, invest_date, sell_date, df):
  invest_rate = list(exchange_rate_30724.values())
  sell_rate = list(exchange_rate_7824.values())

  bought_currencies = exchange_from_usd(portfolio, invest_rate, budget)
  investment_return = exchange_to_usd(bought_currencies, sell_rate)

  profit = investment_return - budget
  return_percent = profit/budget

  return investment_return, profit, return_percent

In [ ]:
filenames = ['AUDUSD_D1.csv', 'BTCUSD_D1.csv', 'ETHUSD_D1.csv', 'EURUSD_D1.csv', 'GBPUSD_D1.csv', 'NZDUSD_D1.csv', 'USDCAD_D1.csv', 'USDCHF_D1.csv', 'USDJPY_D1.csv', 'XAGUSD_D1.csv', 'XAUUSD_D1.csv', ]

header = []
data = []
for name in filenames:
  data.append(pd.read_csv(name, sep='\t'))


In [ ]:
def print_portfolio(portfolio):
  for x in portfolio:
    print(x, portfolio[x])

In [ ]:
portfolio_sharpe = get_good_dict(result_sharpe.x, currencies)

In [ ]:
print(invest_portfolio(portfolio_sharpe, 1000, '7/30/24', '8/5/24', data))
print_portfolio(portfolio_sharpe)
print(f"Risk {mp.pV(result_sharpe.x, retM)}, Expected Profit: {mp.per(result_sharpe.x, retM)}")

(1038.078344997692, 38.07834499769206, 0.038078344997692054)
AUDUSD_D1 0
BTCUSD_D1 0
ETHUSD_D1 0.04060031562752621
EURUSD_D1 0
GBPUSD_D1 0
NZDUSD_D1 0
USDCAD_D1 0.035806194176242395
USDCHF_D1 0
USDJPY_D1 0.9235934901962315
XAGUSD_D1 0
XAUUSD_D1 0
Risk 0.006130792990536765, Expected Profit: 0.0581718571658748


In [ ]:
portfolio_min_risk = get_good_dict(result.x, currencies)
print(invest_portfolio(portfolio_min_risk, 1000, '7/30/24', '8/5/24', data))
print_portfolio(portfolio_min_risk)
print(f"Risk {mp.pV(result.x, retM)}, Expected Profit: {mp.per(result.x, retM)}")

(1018.0532806426618, 18.053280642661775, 0.018053280642661776)
AUDUSD_D1 0
BTCUSD_D1 0
ETHUSD_D1 0
EURUSD_D1 0.1695160379180731
GBPUSD_D1 0
NZDUSD_D1 0
USDCAD_D1 0.3795512352012316
USDCHF_D1 0.19678840869529818
USDJPY_D1 0.2541443181853972
XAGUSD_D1 0
XAUUSD_D1 0
Risk 0.0025080503887368708, Expected Profit: 0.0200915025620585


In [ ]:
portfolio_max_profit = get_good_dict(result_profit_max.x, currencies)
print(invest_portfolio(portfolio_max_profit, 1000, '7/30/24', '8/5/24', data))
print_portfolio(portfolio_max_profit)
print(f"Risk {mp.pV(result_profit_max.x, retM)}, Expected Profit: {mp.per(result_profit_max.x, retM)}")

(709.6445177741111, -290.35548222588886, -0.29035548222588886)
AUDUSD_D1 0
BTCUSD_D1 0
ETHUSD_D1 1
EURUSD_D1 0
GBPUSD_D1 0
NZDUSD_D1 0
USDCAD_D1 0
USDCHF_D1 0
USDJPY_D1 0
XAGUSD_D1 0
XAUUSD_D1 0
Risk 0.6370975461864719, Expected Profit: 0.2608808502783416


In [ ]:
portfolio_maxp_minv = get_good_dict(result_maxp_minv.x, currencies)
print(invest_portfolio(portfolio_maxp_minv, 1000, '7/30/24', '8/5/24', data))
print_portfolio(portfolio_maxp_minv)
print(f"Risk {mp.pV(result_maxp_minv.x, retM)}, Expected Profit: {mp.per(result_maxp_minv.x, retM)}")

(995.4215433125166, -4.578456687483367, -0.004578456687483367)
AUDUSD_D1 0
BTCUSD_D1 0
ETHUSD_D1 0.1707432199013493
EURUSD_D1 0
GBPUSD_D1 0
NZDUSD_D1 0
USDCAD_D1 0
USDCHF_D1 0
USDJPY_D1 0.8292567800986493
XAGUSD_D1 0
XAUUSD_D1 0
Risk 0.0229000260432247, Expected Profit: 0.08685768912848649
